In [2]:
import csv, io, re, pathlib as pl
import numpy as np
import pandas as pd

# ---- paths ----
raw_path = pl.Path("../data/yields_raw.csv")      
out_path = pl.Path("../data/yields_clean.csv")   

# ---- read all non-empty lines robustly ----
lines = raw_path.read_text(encoding="utf-8", errors="ignore").splitlines()

def split_csv_line(line):
    return next(csv.reader(io.StringIO(line)))

rows = []
for ln in lines:
    if ln.strip():
        try:
            rows.append(split_csv_line(ln))
        except Exception:
            # skip malformed footnote lines
            continue

# ---- locate key header rows ----
idx_geo  = next((i for i, r in enumerate(rows) if r and str(r[0]).strip().lower() == "geography"), None)
idx_disp = next((i for i, r in enumerate(rows) if r and str(r[0]).strip().lower().startswith("harvest disposition")), None)
idx_type = next((i for i, r in enumerate(rows) if r and str(r[0]).strip().lower().startswith("type of crop")), None)

if None in (idx_geo, idx_disp, idx_type):
    raise RuntimeError(f"Could not find expected header rows. Found: Geography={idx_geo}, Harvest disposition={idx_disp}, Type of crop={idx_type}")

# ---- extract measure / years ----
measure = (rows[idx_disp][1] if len(rows[idx_disp]) > 1 else "").strip()
year_labels = [str(x).strip() for x in rows[idx_type][1:] if str(x).strip()]
# keep only columns that look like years
year_labels = [y for y in year_labels if re.search(r"\d{4}", y)]

# ---- collect data rows until blank/footnotes ----
data_rows = []
for r in rows[idx_type+1:]:
    c0 = str(r[0]).strip() if r else ""
    if (not r) or (not c0) or c0.startswith("...") or c0.startswith('"'):
        break
    data_rows.append(r)

if not data_rows:
    raise RuntimeError("No data rows detected under 'Type of crop'.")

# ---- build wide dataframe: crop + years ----
max_cols = max(len(r) for r in data_rows)
norm_rows = [r + [None]*(max_cols-len(r)) for r in data_rows]
dfw = pd.DataFrame(norm_rows)
dfw = dfw.iloc[:, :1+len(year_labels)].copy()
dfw.columns = ["crop_raw"] + year_labels[:dfw.shape[1]-1]

# ---- clean crop names (strip trailing reference codes like ' 10 11') ----
def clean_crop_name(s):
    s = str(s).strip().strip('"')
    s = re.sub(r"\s+\d+(?:\s+\d+)*$", "", s)  # remove trailing numeric refs
    return s

dfw["crop"] = dfw["crop_raw"].map(clean_crop_name)

# ---- melt to tidy (year, crop, value) ----
value_cols = [c for c in dfw.columns if c not in ("crop_raw", "crop")]
df_long = dfw.melt(id_vars=["crop"], value_vars=value_cols,
                   var_name="year_str", value_name="value_raw")

# coerce year and numeric value
df_long["year"] = df_long["year_str"].astype(str).str.extract(r"(\d{4})").astype(int)

def to_numeric(x):
    if pd.isna(x): return np.nan
    s = str(x).replace(",", "").strip()
    s = re.sub(r"[^\d\.\-]", "", s)
    try:
        return float(s) if s != "" else np.nan
    except:
        return np.nan

df_long["value"] = df_long["value_raw"].map(to_numeric)

tidy = (df_long
        .dropna(subset=["value"])
        .loc[:, ["year", "crop", "value"]]
        .sort_values(["crop", "year"])
        .reset_index(drop=True))

# ---- name the column if it's yield in tonnes/ha, else keep measure ----
is_yield_mt_ha = bool(re.search(r"yield.*tonnes.*hectare", measure, flags=re.I))
if is_yield_mt_ha:
    tidy = tidy.rename(columns={"value": "yield_mt_ha"})
else:
    tidy["measure"] = measure  # e.g., "Seeded area (acres)"

# ---- save ----
out_path.parent.mkdir(parents=True, exist_ok=True)
tidy.to_csv(out_path, index=False)
print(f"✅ Saved cleaned file to {out_path.resolve()}")
print(f"Detected measure: {measure!r} (yield_mt_ha? {is_yield_mt_ha})")
display(tidy.head())


✅ Saved cleaned file to /Users/trtmacmini2/Desktop/Projects/Agri-Risk-Dashboard/data/yields_clean.csv
Detected measure: 'Seeded area (acres)' (yield_mt_ha? False)


,year,crop,value,measure
0,2016,Barley,2475000.0,Seeded area (acres)
1,2017,Barley,2325000.0,Seeded area (acres)
2,2018,Barley,2692000.0,Seeded area (acres)
3,2019,Barley,3150000.0,Seeded area (acres)
4,2020,Barley,3124400.0,Seeded area (acres)
